In [1]:
import cv2
import numpy as np
import glob

In [2]:
def rotate_image(image):
    # Step 1: Find the centroid of the particle
    M = cv2.moments(image)
    cx = int(M['m10'] / M['m00'])
    cy = int(M['m01'] / M['m00'])

    # Step 2: Find the principal axis of the particle
    _, _, angle = cv2.fitEllipse(np.argwhere(image == 0))
    if angle > 90:
        angle -= 180
    angle = -angle

    # Step 3: Rotate the image to align the principal axis horizontally
    rows, cols = image.shape
    M = cv2.getRotationMatrix2D((cx, cy), angle, 1)
    rotated = cv2.warpAffine(image, M, (cols, rows))

    # Step 4: Reverse the image if the length under the principal axis is longer
    _, _, w, h = cv2.boundingRect(np.argwhere(rotated == 0))
    if h > w:
        rotated = cv2.rotate(rotated, cv2.ROTATE_180)

    return rotated

In [3]:
for filename in glob.glob('*.png'):
    image = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    _, binary = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)
    rotated = rotate_image(binary)
    cv2.imwrite(f'rotated_{filename}', rotated)